In [18]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA

In [19]:
df = pd.read_csv('../../train_merged.csv', sep=',', header='infer')

In [20]:
ts = df[(df.Store == 1) & (df.Dept == 17)]
ts = ts[['Weekly_Sales', 'Date']]
ts = ts.set_index('Date')

In [21]:
std =  ts.rolling(window=52).std()
mean = ts.rolling(window=52).mean()
ewma = ts.ewm(halflife=12).mean()

In [23]:
ts_diff = ts - mean
ts_diff.dropna(inplace=True)

In [24]:
ts_diff = ts - ewma
ts_diff.dropna(inplace=True)

In [25]:
model = ARIMA(ts_diff, order=(0, 1, 2))  
results_ARIMA = model.fit()

yhat = results_ARIMA.fittedvalues
yhat *= -1

In [26]:
from datetime import datetime
import math

end_date = datetime.strptime(ts.index[-1], '%Y-%m-%d')

def eval_weeknum(x):
    return math.ceil((x - end_date).days / 7)

test = pd.read_csv('../../data/sampleSubmission.csv', sep='_')
test['Date'] = pd.to_datetime(test['Date'])
test

,Store,Dept,Date
0,1,1,2012-11-02
1,1,1,2012-11-09
2,1,1,2012-11-16
3,1,1,2012-11-23
4,1,1,2012-11-30
5,1,1,2012-12-07
6,1,1,2012-12-14
7,1,1,2012-12-21
8,1,1,2012-12-28
9,1,1,2013-01-04


In [27]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return np.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]


def predict (store, dept, weeknum):
    ts = df[(df.Store == store) & (df.Dept == dept)]
    ts = ts[['Weekly_Sales', 'Date']]
    ts = ts.set_index('Date')
    
    series = ts
    # seasonal difference
    X = series.values
    weeks_in_year = 52
    differenced = difference(X, weeks_in_year)
    # fit model
    model = ARIMA(differenced, order=(0,1,2))
    model_fit = model.fit(disp=0)
    # multi-step out-of-sample forecast

    start_index = len(differenced)
    end_index = start_index + weeknum - 1

    forecast = model_fit.predict(start=start_index, end=end_index)

    # invert the differenced forecast to something usable
    history = [x for x in X]
    for yhat in forecast:
        inverted = inverse_difference(history, yhat, weeks_in_year)
        history.append(inverted)

    inverse = []
    for yhat in history:
        inverse.append(yhat[0])
        
    return inverse[-1]

In [43]:
import sys

test['Weekly_Salse'] = 0
defualt_end_date = end_date
for index, row in test.iterrows():
    sys.stdout.write('\r'+str((index/test.shape[0])*100))
    
    try:
        ts = df[(df.Store==row.Store) & (df.Dept==row.Dept)]
        ts = ts[['Weekly_Sales', 'Date']]
        ts = ts.set_index('Date')
        end_date = datetime.strptime(ts.index[-1], '%Y-%m-%d')
    except:
        end_date = defualt_end_date
    try:
        test.loc[index, 'Weekly_Salse'] = predict(row['Store'], row['Dept'], eval_weeknum(row['Date']))
    except:
        test.loc[index, 'Weekly_Salse'] = 0

0.203365083779543

C:\Users\KritsanaCHAIKEAW\Anaconda3\lib\site-packages\statsmodels\base\model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


6.999582840855

C:\Users\KritsanaCHAIKEAW\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:628: RuntimeWarning: overflow encountered in exp
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\Users\KritsanaCHAIKEAW\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:628: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\Users\KritsanaCHAIKEAW\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:629: RuntimeWarning: overflow encountered in exp
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
C:\Users\KritsanaCHAIKEAW\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:629: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()


7.03260793993

...

89.36157269

C:\Users\KritsanaCHAIKEAW\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\KritsanaCHAIKEAW\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


89.3641799346

C:\Users\KritsanaCHAIKEAW\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


99.9991309184

In [72]:
test
submit = pd.read_csv('../../data/sampleSubmission.csv')
submit['Weekly_Sales'] = test['Weekly_Salse']
submit.columns = ['Id', 'Weekly_Sales']
submit.to_csv('../../data/ARIMA_submit.csv', index=False, na_rep='0')

In [75]:
test.shape[0]/test[test['Weekly_Salse']==0].shape[0]

4.085934448350556

In [89]:
df['WeekNum'] = pd.to_datetime(df['Date']).dt.week
test_copy = test.copy()

for index, row in test_copy[test_copy['Weekly_Salse']==0].iterrows():
    s = row['Store']
    d = row['Dept']
    w = row['Date'].week
    old = df[(df.Store==s) & (df.Dept==d) & (df.WeekNum==w)]['Weekly_Sales']
    test_copy.loc[index, 'Weekly_Salse'] = old.mean()
test_copy

,Store,Dept,Date,Weekly_Salse
0,1,1,2012-11-02,40979.315851
1,1,1,2012-11-09,18701.931504
2,1,1,2012-11-16,19063.051504
3,1,1,2012-11-23,20923.641504
4,1,1,2012-11-30,25305.881504
5,1,1,2012-12-07,33318.311504
6,1,1,2012-12-14,45785.421504
7,1,1,2012-12-21,46801.141504
8,1,1,2012-12-28,23363.271504
9,1,1,2013-01-04,16580.081504


In [91]:
submit_copy = pd.read_csv('../../data/sampleSubmission.csv')
submit_copy['Weekly_Sales'] = test_copy['Weekly_Salse']
submit_copy.columns = ['Id', 'Weekly_Sales']
submit_copy.to_csv('../../data/ARIMA_submit_copy.csv', index=False, na_rep='0')